<a href="https://colab.research.google.com/github/benbaz-2/comp551/blob/main/CNN_task2.6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [1]:
!pip install medmnist

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=6886bd7a236e00bb3f41ed97b5aaf86fc58d018bb82b4827628f08744adeb66a
  Stored in directory: /root/.cache/pip/wheels/19/39/2f/2d3cadc408a8804103f1c34ddd4b9f6a93497b11fa96fe738e
Successfully built fire


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
from medmnist import OrganAMNIST
from tqdm import tqdm

In [3]:
import os
import numpy as np
import shutil
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
from matplotlib import pyplot
import matplotlib.image as mpimg

import torch
import torch.nn as nn
from torchvision.datasets import MNIST, FashionMNIST
from torchvision.transforms import ToTensor, Compose, Normalize, RandomHorizontalFlip, RandomRotation

## Define transforms

In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

## Load data

In [5]:
train_dataset = OrganAMNIST(split='train', download=True, transform=transform)
test_dataset = OrganAMNIST(split='test', download=True, transform=transform)
val_dataset = OrganAMNIST(split='val', download=True, transform=transform)

100%|██████████| 38.2M/38.2M [00:03<00:00, 9.81MB/s]


Using downloaded and verified file: /root/.medmnist/organamnist.npz
Using downloaded and verified file: /root/.medmnist/organamnist.npz


In [6]:
# Use dataloaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

## Model implementation

In [7]:
num_filters = 8
filter_size = 3
pool_size   = 2
num_classes = 10
batch_size  = 64
strides     = 1
padding     = 1  # Padding of 1 with filter size of 3 output the same dimension
dropout_rate = 0.25

class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential()
        self.model.append(nn.Conv2d(1, num_filters, filter_size, strides, padding))

        self.model.append(nn.MaxPool2d(pool_size))

        self.model.append(nn.Conv2d(num_filters,  num_filters ** 2, filter_size, strides, padding))
        self.model.append(nn.ReLU())
        self.model.append(nn.MaxPool2d(pool_size))

        self.model.append(nn.Dropout(dropout_rate)) # randomly sets input units to 0 with a frequency of rate at each step during training time, to avoid overfitting
        self.model.append(nn.Flatten()) # Flatten serves as a connection between the convolution and dense layers
        self.model.append(nn.Linear(3136, 1000)) # hidden layer
        self.model.append(nn.ReLU())
        self.model.append(nn.Dropout(dropout_rate))
        self.model.append(nn.Linear(1000, num_classes))
        self.model.append(nn.Softmax(dim=1))

    def forward(self, x):
        return self.model(x)


## Model Training

In [8]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
model = CNN()
model.to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
num_epochs = 10

In [9]:
def compute_accuracy(model = model, data_loader = train_loader, device=device):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():  # Disable gradient calculation for evaluation
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)  # Get logits
            _, predicted = torch.max(outputs.data, 1)  # Get predicted class indices

            total += labels.size(0)  # Total number of labels
            correct += (predicted == labels).sum().item()  # Count correct predictions

    accuracy = correct / total * 100  # Convert to percentage
    return accuracy

In [10]:

def train(model=model, optimizer=optimizer, train_loader=train_loader, val_loader=val_loader, criterion=criterion, num_epochs=num_epochs, device=device):

    # Training loop
    history = {"loss": [], "accuracy": [], "val_accuracy": []}

    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        running_loss = 0.0

        for images, labels in tqdm(train_loader):
            # Move tensors to the configured device (GPU or CPU)
            images, labels = images.to(device), labels.to(device)

            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(images)

            # Calculate loss
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # Print the average loss for the epoch
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')
        history["loss"].append(running_loss/len(train_loader))
        history["accuracy"].append(compute_accuracy(model, train_loader, device))
        history["val_accuracy"].append(compute_accuracy(model, test_loader, device))

    print('Training complete!')
    return history



In [11]:
history = train()

!pip install medmnist
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
from medmnist import OrganAMNIST
from tqdm import tqdm
import os
import numpy as np
import shutil
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
from matplotlib import pyplot
import matplotlib.image as mpimg

import torch
import torch.nn as nn
from torchvision.datasets import MNIST, FashionMNIST
from torchvision.transforms import ToTensor, Compose, Normalize, RandomHorizontalFlip, RandomRotation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
train_dataset = OrganAMNIST(split='train', download=True, transform=transform)
test_dataset = OrganAMNIST(split='test', download=True, transform=transform)
val_dataset = OrganAMNIST(split='val', download=True, transform=transform)
#